In [1]:
#This notebook will read in a source list and download relevent data
#NOTE: this was written on MacOS so uses curl. You may have to change this to wget for other systems.
#Will create a new folder for each source to download the images in

# Emma Alexander November 2021

#----------Setup----------------
#imports
from astropy.io import fits
from astropy.wcs import WCS
import numpy as np 
from astroquery.skyview import SkyView
import os
import astropy.units as u
from astropy.coordinates import SkyCoord
import warnings
warnings.filterwarnings('ignore')

#read in the file with the sources
#file format should be (1) Name (2) Centre coordinates (3) Image size (arcsec)
rootdirectory='/Volumes/NARNIA/pilot_cutouts/'
sourcefile = '/Users/emma/GitHub/possum-tools/DataProcess/pilot_sources.txt'
sources=np.loadtxt(sourcefile,dtype='str')

# list of survey images you wish to download 
# for names see https://astroquery.readthedocs.io/en/latest/skyview/skyview.html
# or use SkyView.list_surveys()
# change these in and out as you wish
# note that if a survey is not available for the coordinates you specify, then it will download an empty fits
surveys= ['WISE 3.4','WISE 4.6','WISE 12']
#,'DSS','DSS2 Red','DSS2 Blue','DSS2 IR','SUMSS 843 MHz','2MASS-J','GLEAM 170-231 MHz']

# annoyingly, SkyView doesn't register the native pixel size of surveys, so need to specify
# load in the file with these data from https://skyview.gsfc.nasa.gov/current/cgi/survey.pl
survey_pix=np.loadtxt('survey_pix.txt',dtype='str')

# alternatively, specify a common reolution you would like all the surveys to be sampled at
# beware resampling errors... 
commonpixscale=False
pixscale=1 #arcseconds

#specify if you want to overwrite existing images or not
overwrite=False




In [ ]:
# begin finding and downloading data
# loop over sources in source list
for i in range (0,sources.shape[0]):
    
    # extract source name from source list
    src=sources[i,0]
    # create a folder for the source if one doesn't exist already
    directory=rootdirectory+src+'/'
    
    if os.path.isdir(directory)==False:
        os.system('mkdir '+directory)
        print("Making directory for source "+src)
        
    #get source coords and download image
    #coordsstr=sources[i,1]
    #coords = SkyCoord(coordsstr)   
    ra=sources[i,1]
    dec=sources[i,2]
    coords=SkyCoord(ra,dec)
    
    imsize=0.8*float(sources[i,6])
    
    #loop over surveys
    for j in range (0,len(surveys)):
        survey=surveys[j]
        survey_nospace=survey.replace(" ","_")
        
        if commonpixscale==False:
        #get the pixel scale of the survey
        #otherwise will use the value already specified in the cell above
            idx=np.where(survey_pix==survey_nospace)
            try:
                pixscale=float(survey_pix[idx[0][0],1])
            except:
                print("Error: no pixscale specified in file for {}".format(survey_nospace))
                break
            
        # make the image name
        imagename=src+'_'+survey_nospace+'.fits'
        
        n_pix=int(imsize/pixscale)
        
        #only download if file doesn't already exist 
        if (os.path.isfile(directory+imagename)==False) or (overwrite==True):
            try:
                url=SkyView.get_image_list(position=coords,survey=[survey],coordinates='J2000',projection='Sin',resolver='NED',pixels=n_pix,height=imsize*u.arcsec,width=imsize*u.arcsec)
                os.system('curl -o '+directory+imagename+' '+url[0])
                print("Downloaded {} image for source {}".format(survey,src))
            except:
                print("Survey {} not found in SkyView".format(survey))
                
        elif (os.path.isfile(directory+imagename)==True):
            b = os.path.getsize(directory+imagename)
            if b<5000:
                tries=0
                print(b)
                while b<=5000 and tries<3:   
                    try:
                        url=SkyView.get_image_list(position=coords,survey=[survey],coordinates='J2000',projection='Sin',resolver='NED',pixels=n_pix,height=imsize*u.arcsec,width=imsize*u.arcsec)
                        os.system('curl -o '+directory+imagename+' '+url[0])
                        print("Downloaded {} image for source {}".format(survey,src))
                        tries+=1
                        print(b)
                    except:
                        print("somethign went wrong")
        
        else:
             print("Not downloading {} image for source {}".format(survey,src))      

4197
Downloaded WISE 3.4 image for source 2052-5246
4197
Downloaded WISE 3.4 image for source 2052-5246
4197
Downloaded WISE 3.4 image for source 2052-5246
4197
4197
Downloaded WISE 4.6 image for source 2052-5246
4197
Downloaded WISE 4.6 image for source 2052-5246
4197
Downloaded WISE 4.6 image for source 2052-5246
4197
4197
Downloaded WISE 12 image for source 2052-5246
4197
Downloaded WISE 12 image for source 2052-5246
4197
Downloaded WISE 12 image for source 2052-5246
4197
4197
Downloaded WISE 3.4 image for source 2129-5053
4197
Downloaded WISE 3.4 image for source 2129-5053
4197
Downloaded WISE 3.4 image for source 2129-5053
4197
4197
somethign went wrong
somethign went wrong
somethign went wrong
somethign went wrong
somethign went wrong
somethign went wrong
somethign went wrong
somethign went wrong
somethign went wrong
somethign went wrong
somethign went wrong
somethign went wrong
somethign went wrong
somethign went wrong
somethign went wrong
somethign went wrong
somethign went wro